# Regresja cen nieruchomości

Zadanie polega na predykcji cen nieruchomości na podstawie cech takich jak współrzędne geograficzne, parametry techniczne (np. liczba sypialni, łazienek, pięter) oraz oceny (np. standardu wykończenia czy stanu nieruchomości).

Dane: https://www.kaggle.com/shivachandel/kc-house-data

## Kroki, które należy wykonać w celu reprodukcji rozwiązania:
* Utworzenie zasobu Machine Learning
* Utworzenie zasobu obliczeniowego (Compute)
* Załadowanie zbioru danych do Azure Machine Learning
* Przygotowanie kodu

## Przygotowanie potrzebnych bibliotek i wczytane Workspace'u


In [6]:
    from azureml.core.workspace import Workspace
    from azureml.core.dataset import Dataset
    from azureml.core.experiment import Experiment
    from azureml.train.automl import AutoMLConfig
    from azureml.widgets import RunDetails
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error

In [7]:
ws = Workspace.from_config()

## Wczytanie danych

In [8]:
dataset = Dataset.get_by_name(workspace=ws, name="kc_house_data")

## Przekształcenie danych w ramkę i prezentacja kilku wierszy dancyh

In [9]:
df = dataset.to_pandas_dataframe()
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180.0,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170.0,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770.0,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050.0,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680.0,0,1987,0,98074,47.6168,-122.045,1800,7503


## Podział zbioru na dane uczące i testowe

In [10]:
df_train, df_test = train_test_split(df, test_size=0.2)

## Przygotowanie ustawień eksperymentu

In [11]:
settings = {
    "iteration_timeout_minutes": 3,
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "primary_metric": 'normalized_root_mean_squared_error',
    "featurization": 'auto',
    "n_cross_validations": 4,
    "label_column_name":"price",# nazwa kolumny ze zmienną opisywaną
    "task":"regression",
    "iterations": 20, #liczba iteracji do wykonania
    "max_cores_per_iteration": -1, #Wszystkie dostępne rdzenie
}

experiment_config = AutoMLConfig(training_data=df_train, **settings)

## Uruchamianie eksperymentu

In [12]:
experiment = Experiment(ws, "PWAutoMLFinalExperiment")
experiment_result = experiment.submit(experiment_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_736718c1-f7f7-413d-ad6b-78121c710292

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type a

Cannot serialize JSON, possibly due to NaN or Inf, scrubbing to zero and retrying...


## Prezentacja wyników

## Wczytanie najlepszego modelu

In [16]:
best_run, fitted_model = experiment_result.get_output()

## Predykcja na zbiorze testowym i pomiar jakości wyników

In [17]:
X_test = df_test.drop("price", axis=1)
y_test = df_test["price"]
y_pred = fitted_model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root mean squared error: {rmse_test}")

Root mean squared error: 130578.41935839684
